In [1]:
from pprint import pprint

## Local Search - Genetic Algorithm

For this assignment we're going to use the Genetic Algorithm to find the solution to a shifted Sphere Function in 10 dimensions, $x$, where the range of $x_i$ in each dimension is (-5.12 to 5.12). Here a "solution" means the vector $x$ that minimizes the function. The Sphere Function is:

$$f(x)=\sum x^2_i$$

We are going to shift it over 0.5 in every dimension:

$$f(x) = \sum (x_i - 0.5)^2$$

where $n = 10$.

As this *is* a minimization problem you'll need to use the trick described in the lecture to turn the shifted Sphere Function into an appropriate fitness function (which is always looking for a *maximum* value).

<div style="background: palegreen; margin:20px; padding: 20px;">
    <strong>Question</strong>
    <p>
<strong>This is not supposed to be a hard problem but an illustrative one.</strong> If the minimizing vector for the regular sphere function is all 0.0, what is the minimizing vector of values for this shifted shifted sphere function in 10 dimensions?
</p>
</div>

the minimizing vector would be all 0.5.

GA can be used solve problems like these in multiple dimensions (the 2d version is shown):

![Eggholder](http://www.sfu.ca/~ssurjano/egg.png)

## Canonical ("Binary") Genetic Algorithm

You are going to solve the problem two different ways. First, using the traditional (or "Canonical") Genetic Algorithm that encodes numeric values as binary strings (you don't have to represent them literally as strings but they are general lists or strings of only 0 or 1).

There are many different ways to make this encoding. For this assignment, use a 10 bit binary encoding for each $x_i$. We can then *decode* each $x_i$ a to an integer value of 0 to 1024 and then *approximately* map that to (-5.12, 5.12) by subtracting 512 and dividing by 100. If the boundaries mattered for your problem, you would need to improve your encoding!

All the GA operators (crossover, mutation) should be as described in the lecture and pseudocode.


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
Please remember that there is a difference between the *genotype* and the *phenotype*. The Canonical GA operates on the *genotype* (the encoding) and does not respect the boundaries of the phenotype (the decoding). So, for example, do <strong>not</strong> use a List of Lists to represent an individual. It should be a <strong>single</strong> List of bits. How many bits depends on the encoding and the number of variables encoded. In general, crossover and mutation have no idea what those bits represent so boundaries between variables are not respected in any way.</p>
    <p>Also remember...
        <ul>
            <li>Every individual is a complete solution all by itself. If the problem has 5 variables, the individual has 5 variables encoded in it.</li>
            <li>A population is a collection of individuals, that is, potential solutions.</li>
            <li>Population size and generations are somewhat fungible...you can have a "smaller" population and have it run more generations or vice versa but the substitution is not exact. Experiment</li>
    </ul>
</div>

## Real Valued GA

For the real valued GA, you can represent each $x_i$ as a float in the range (-5.12, 5.12) but you will need to create a new mutation operator that applies gaussian noise. Python's random number generator for the normal distribution is called `gauss` and is found in the random module:

```
from random import gauss, random
```

You may need to experiment a bit with the standard deviation of the noise but the mean will be 0.0.

## GA

The GA has a lot of parameters: mutation rate, crossover rate, population size, dimensions (given for this problem), number of generations.  You can put all of those and your fitness function in a `Dict` in which case you need to implement:

```python
def binary_ga( parameters, debug=False):
  pass
```

and

```python
def real_ga( parameters, debug=False):
  pass
```

Remember that you need to transform the sphere function (minimization) into a legit fitness function (maximization). Since you also need the sphere function, I would suggest that your parameters Dict includes something like:

```python
parameters = {
   "f": lambda xs: sphere( 0.5, xs),
   "minimization": True
   # put other parameters in here.
}
```

and then have your code check for "minimization" and create an entry for "fitness" that is appropriate.

Each algorithm may require different values for the parameters. You will need to find a good population size but it is often in the 100s. The number of generations required is also in the 100s. Not 10, not 5.

The Genetic Algorithm itself will have the same basic structure in each case: create a population, evaluate it, select parents, apply crossover and mutation, repeat until the number of desired generations have been generated. The easiest way to accomplish this in "Functional" Python would be to use Higher Order Functions. If you do it this way, you'd have a `general_ga` that the functions above might call along with specialization functions. If you have no idea what I'm taking about...don't do it! It's not required.

The Binary GA should return the following Dict:

```
{
    "genotype": [0, 1, ....],
    "phenotype": [0.59, 0.47, 0.5,....],
    "solution": [0.59, 0.47, 0.5,....],
    "fitness": 0.997,
    "f": 0.001
}
```

The values are representative. Yes, the solution is the same as phenotype in this case.

The Real Valued GA should return a similar Dict except there is no "phenotype":
```
{
    "solution": [0.59, 0.47, 0.5,....],
    "fitness": 0.997,
    "f": 0.001
}
```

Since the phenotype and the genotype are the same, we don't need to show them separately.

Use `pprint` on the returned value.

**Additionally**, if the formal argument `debug` is set to true, you should print the current best individual of every N generations. Print out the same information as above plus the generation number. N should be about your generations/20 (so there are 20 debug outputs).

In [3]:
def sphere( shift, xs):
    return sum( [(x - shift)**2 for x in xs])

In [4]:
sphere( 0.5, [1.0, 2.0, -3.4, 5.0, -1.2, 3.23, 2.87, -4.23, 3.82, -4.61])

113.42720000000001

-----

Put your code in cells here (not all in one cell! One cell per function. Make additional cells (code and Markdown) as needed.

In [533]:
# helpers and implementation

This function randomly generates an initial population of 10 individuals.  Each variable in each individual is randomly assigned a 10 bit binary number

In [630]:
import random
def generate_population():
    initial_population = []
    for i in range(10):
        parent = ''
        for i in range(10):
            key1 = "" 
            for i in range(10):
                temp = str(random.randint(0, 1)) 
                key1 += temp 
            parent+=key1
        initial_population.append(parent)
    return(initial_population) 

This function decodes a 10 bit binary number (or an individual xi variable), then subtracts 512 and divides by 100 for our desired sphere.  It is a helper function used in decode_parent_array, which decodes an entire individual.

In [632]:
def decode(xi):
    decoded_xi = (int(xi, 2)-512)/100
    return(decoded_xi)

This function evaluates the fitness of each individual.  It looks at all 10 variables contained in each individual, and gets the difference between the current xi values, and the desired value of .5 for all xi.

In [633]:
def fitness(prospect):
    sum_array = []
    for numbers in prospect:
        difference = (numbers-.5)**2
        sum_array.append(difference)
    Sum = sum(sum_array)
    return Sum

This function decodes an individual to 10 binary values.  Each individual is composed of 10 variables, each 10 bits.  Each of these 10 bit variables is converted to binary.

In [634]:
def decode_parent_array(parent_array):    
    selected_parents = []
    for parents in parent_array:
        xi = ''
        counter = 0
        parent = []
        for bits in parents:
            xi+= bits
            counter+=1
            if counter == 10:
                parent.append(decode(xi))
                counter = 0
                xi = ''
        selected_parents.append(parent)  
    return selected_parents

This function selects which parents from the population it should breed.  It returns the two individuals with the best fitness score, which are then bred in the crossover function.

In [635]:
def selection(population, realGA):
    sum_array = []
    possible_parents = []    
    if realGA == True:
        possible_parents = population
    else:
        possible_parents = decode_parent_array(population) 
    for parents in possible_parents:
        Sum = fitness(parents)
        sum_array.append(Sum)
    sorted_sums = sorted(sum_array)
    counter = -1
    breeding_parents = []
    for i in sum_array:
        counter+=1
        if i == sorted_sums[0]:
            breeding_parents.append(population[counter])
            break
    counter = -1
    for j in sum_array:
        counter+=1
        if j == sorted_sums[1]:        
            breeding_parents.append(population[counter])
            break
    return(breeding_parents)      

This function breeds the 2 best individual parents selected from the population.  It selects the midpoint of each parent as a pivot point, and pairs the first half of the first parent with the second half of the second parent, and the second half of the first parent with the first half of the second parent

In [636]:
#pick gene location at random
def crossover(parents, mutation = False):
    parent_0 = parents[0]
    parent_1 = parents[1]
    if mutation == True:
        mutation_func(parent_0, parent_1)
    crossover_point = (len(parent_1)/2)
    crossover_point = int(crossover_point)
    parent_0_son = parent_0[:crossover_point]
    parent_0_daughter = parent_0[crossover_point:]
    parent_1_son = parent_1[:crossover_point]
    parent_1_daughter = parent_1[crossover_point:]
    son = parent_0_son + parent_1_son
    daughter = parent_0_daughter + parent_1_daughter
    return son, daughter   

This function brings all the previous functions together and runs on a population for 1000 generations.  It initially generates a random population, then picks the best parents, breeds them, and adds the child to the population.  It then continues doing so for 1000 generations.

In [667]:
def binary_ga(parameters, debug=False):      
    pop = generate_population()
    generations = 100
    limit = 0
    counter = 0
    while limit < generations:
        breeding_parents = selection(pop, False)
        son, daughter = crossover(breeding_parents)
        pop.append(son)
        pop.append(daughter)
        limit+=1
        if debug == True:
            counter +=1
        if debug ==True and counter ==20:
            a=0
    breeders = selection(pop, False)
    selected_parents = []
    selected_parents_decoded = decode_parent_array(breeders)
    return ("genotype: ", breeders[0]), ("phenotype: ", selected_parents_decoded[0]), ("solution: ", selected_parents_decoded[0]), ("f", fitness(selected_parents_decoded[0]))

The real ga algorithm requires non binary population, so I created a different function to create the initial population.  It creates 10 individuals, with 10 variables randomly selected in the range {-5.12, 5.12}

In [668]:
def generate_population_realga():
    initial_population = []
    for i in range(10):
        parent = []
        for i in range(10):
            key1 = random.uniform(-5.12, 5.12)
            parent.append(key1)
        initial_population.append(parent)
    return(initial_population) 

In the real ga algorithm, we must apply a mutation to the parents.  Here, we apply gaussian noise to a random variable in each parent to slightly adjust their values.

In [669]:
from random import gauss
def mutation_func(parent1, parent2):
    value1 = gauss(-5.12, 5.12)
    value2 = gauss(-5.12, 5.12)
    index1 = random.randint(0,8)
    index2 = random.randint(0,8)
    parent1[index1] = value1
    parent2[index2] = value2   

The real ga algorithm is very similar to the binary ga algorithm, however, it doesn't have to worry about decoding binary values, though it does have to make mutations to parents.  

In [670]:
def real_ga(parameters, debug=False):
    pop = generate_population_realga()
    generations = 100
    limit = 0
    counter = 0
    while limit < generations:
        breeding_parents = selection(pop, True)
        son, daughter = crossover(breeding_parents, True)
        pop.append(son)
        pop.append(daughter)
        limit+=1
        if debug == True:
            counter +=1
        if debug ==True and counter ==20:
            a=0           
    breeders = selection(pop, True)
    return ("solution: ", breeders[0]) , ("f", fitness(breeders[0]))

### Canonical GA

In [671]:
parameters1 = {
    
}
result1 = binary_ga(parameters1, debug=True)
pprint(result1)

(('genotype: ',
  '0110111111101000000001111110011000000000100001111001101111111010000000011111100110000000001000011110'),
 ('phenotype: ', [-0.65, 1.28, -0.07, 0.0, 0.3, -0.65, 1.28, -0.07, 0.0, 0.3]),
 ('solution: ', [-0.65, 1.28, -0.07, 0.0, 0.3, -0.65, 1.28, -0.07, 0.0, 0.3]),
 ('f', 5.0916))


### Real Valued GA

In [672]:
parameters2 = {
    
}
result2 = real_ga(parameters2, debug=True)
pprint(result2)

(('solution: ',
  [-1.6068151151808832,
   -0.2939646855085938,
   -4.128369654436808,
   -5.6698658383756575,
   -0.2266256423358648,
   0.396889574730686,
   -0.2939646855085938,
   1.959777904453774,
   1.8198052434318486,
   -0.2266256423358648]),
 ('f', 70.1279187138115))
